In [1]:

%run ../../load_magic/storage.py
%run ../../load_magic/dataframes.py
from bs4 import BeautifulSoup
%pprint
%who

DATA_FOLDER: ../data/
SAVES_FOLDER: ../saves/
Pretty printing has been turned OFF
BeautifulSoup	 DATA_CSV_FOLDER	 DATA_FOLDER	 ENCODING_TYPE	 SAVES_CSV_FOLDER	 SAVES_FOLDER	 SAVES_PICKLE_FOLDER	 attempt_to_pickle	 example_iterrows	 
get_column_descriptions	 load_csv	 load_dataframes	 load_object	 math	 os	 pd	 pickle	 save_dataframes	 
store_objects	 sys	 



# Display the Civilization Bonuses

In [2]:

selector_str = '#Britons > div.Overview > ul > li.bonuses > ul'
xpath_str = '//*[@id="Britons"]/div[1]/ul/li[5]/ul'
html_str = '''<li>
    Civilization bonuses:
    <ul>
        <li>Town Centers cost -50% wood starting in the Castle Age.</li>
        <li>Foot archers (except Skirmishers) have +1/+2 range in the Castle/Imperial Age.</li>
        <li>Shepherds work 25% faster.</li>
    </ul>
</li>'''
html_folder = os.path.join(DATA_FOLDER, 'html')
file_path = os.path.join(html_folder, 'aoe2_civ_bonuses.html')
with open(file_path, 'rb') as f:
    html_doc = f.read().decode('utf-8')

In [3]:

soup = BeautifulSoup(html_doc, 'html.parser')

In [4]:

sub_selector = 'div.Overview > ul > li.bonuses > ul'
civ_name_list = []
for civ_div in soup.find_all('div', id=True):
    civ_name = civ_div.get_attribute_list('id')[0]
    print(civ_name)
    bonuses_ul = civ_div.select(sub_selector)[0]
    for bonus_li in bonuses_ul.find_all('li'):
        if bonus_li.string:
            print(bonus_li.string)
    print()
store_objects(civ_name_list=civ_name_list)

Britons
Town Centers cost -50% wood starting in the Castle Age.
Foot archers (except Skirmishers) have +1/+2 range in the Castle/Imperial Age.
Shepherds work 25% faster.

Byzantines
Buildings have +10%/+20%/+30%/+40% HP in the Dark/Feudal/Castle/Imperial Age.
Camels, Skirmishers, and the Spearman lines are 25% cheaper.
Fire Ships attack 20% faster.
Advancing to the Imperial Age is 33% cheaper.
Town Watch is free.

Celts
Infantry units move 15% faster.
Lumberjacks work 15% faster.
Siege weapons fire +20% faster.
Enemy herdables can be converted regardless of enemy units next to them.

Chinese
Town Center supports ten population (instead of five).
Technologies are 10%/15%/20% cheaper in the Feudal/Castle/Imperial Age.
Demolition Ships have +50% HP.

Franks
Farm upgrades are free.
Castles are 25% cheaper.
Cavalry have +20% HP.
Foragers work 25% faster.

Goths
Infantry are 35% cheaper starting in the Feudal Age.
Infantry have +1 attack bonus against standard buildings.
+10 population cap i

In [10]:

sub_selector = 'div.Overview > ul > li.bonuses > ul'
for civ_div in soup.find_all('div', id=True):
    civ_name = civ_div.get_attribute_list('id')[0]
    bonuses_ul = civ_div.select(sub_selector)[0]
    for bonus_li in bonuses_ul.find_all('li'):
        if bonus_li.string:
            print(bonus_li.string)

Town Centers cost -50% wood starting in the Castle Age.
Foot archers (except Skirmishers) have +1/+2 range in the Castle/Imperial Age.
Shepherds work 25% faster.
Buildings have +10%/+20%/+30%/+40% HP in the Dark/Feudal/Castle/Imperial Age.
Camels, Skirmishers, and the Spearman lines are 25% cheaper.
Fire Ships attack 20% faster.
Advancing to the Imperial Age is 33% cheaper.
Town Watch is free.
Infantry units move 15% faster.
Lumberjacks work 15% faster.
Siege weapons fire +20% faster.
Enemy herdables can be converted regardless of enemy units next to them.
Town Center supports ten population (instead of five).
Technologies are 10%/15%/20% cheaper in the Feudal/Castle/Imperial Age.
Demolition Ships have +50% HP.
Farm upgrades are free.
Castles are 25% cheaper.
Cavalry have +20% HP.
Foragers work 25% faster.
Infantry are 35% cheaper starting in the Feudal Age.
Infantry have +1 attack bonus against standard buildings.
+10 population cap in the Imperial Age.
Fishing Ships have double HP, +

In [11]:

sub_selector = 'div.Overview > ul > li.bonuses > ul'
civ_name_list = []
for civ_div in soup.find_all('div', id=True):
    civ_name = civ_div.get_attribute_list('id')[0]
    civ_name_list.append(civ_name.strip())

In [14]:

import re

line_regex = re.compile('[\r\n]+', re.MULTILINE)
with open(file_path, 'rb') as f:
    feature_list = line_regex.split(f.read().decode('utf-8').strip())

In [15]:

features_df = pd.DataFrame(data=None, index=civ_name_list, columns=feature_list)

In [16]:

csv_folder = os.path.join(DATA_FOLDER, 'csv')
file_path = os.path.join(csv_folder, 'aoe2_civ_features.csv')
features_df.to_csv(file_path)

In [17]:

import urllib.request

civ_soup_dict = {}
for civ_name in civ_name_list:
    try:
        civ_url = 'https://ageofempires.fandom.com/wiki/{}_(Age_of_Empires_II)'.format(civ_name)
        with urllib.request.urlopen(civ_url) as response:
            civ_html = response.read()
    except:
        civ_url = 'https://ageofempires.fandom.com/wiki/{}'.format(civ_name)
        with urllib.request.urlopen(civ_url) as response:
            civ_html = response.read()
    civ_soup = BeautifulSoup(civ_html, 'html.parser')
    civ_soup_dict[civ_name] = civ_soup

In [ ]:

for i, civ_name in enumerate(civ_soup_dict):
    print(civ_name)
    bonuses_selector = '#Civilization_bonuses'
    bonuses_span = civ_soup_dict[civ_name].select(bonuses_selector)[0]
    bonuses_ul = list(bonuses_span.next_elements)[6]
    for bonus_li in bonuses_ul.find_all('li'):
        print(bonus_li.text.strip())
    print()

In [19]:

txt_folder = os.path.join(DATA_FOLDER, 'txt')
file_path = os.path.join(txt_folder, 'aoe2_civ_features.txt')
with open(file_path, 'wb') as f:
    f.write('\n'.encode('utf-8'))
    for i, civ_name in enumerate(civ_soup_dict):
        bonuses_selector = '#Civilization_bonuses'
        bonuses_span = civ_soup_dict[civ_name].select(bonuses_selector)[0]
        bonuses_ul = list(bonuses_span.next_elements)[6]
        for bonus_li in bonuses_ul.find_all('li'):
            f.write('{}\n'.format(bonus_li.text.strip()).encode('utf-8'))


Make sure to clean up the list before running the cell below to load it back in.

In [ ]:

import re

line_regex = re.compile('[\r\n]+', re.MULTILINE)
with open(file_path, 'rb') as f:
    feature_list = line_regex.split(f.read().decode('utf-8').strip())

In [ ]:

space_regex = re.compile('\s+')
feature_list = [space_regex.sub('_', feature.strip()) for feature in feature_list]

In [ ]:

features_df = pd.DataFrame(data=None, index=civ_name_list, columns=feature_list)

In [ ]:

xlsx_folder = os.path.join(DATA_FOLDER, 'xlsx')
os.makedirs(name=xlsx_folder, exist_ok=True)
file_path = os.path.join(xlsx_folder, 'aoe2_civ_features.xlsx')
features_df.to_excel(file_path)